In [1]:
from strategies.BinanceBtcDeapV1Voting.models.config import FEAT_L6, FEAT_FRACDIFF, DeepSSMContainer, LGSSMContainer
from jesse.helpers import date_to_timestamp
import lightgbm as lgb
import pandas as pd
import numpy as np

df_feat_raw = pd.read_parquet("data/feat_hard_L6.parquet")

model_deep_ssm = DeepSSMContainer()
model_lg_ssm = LGSSMContainer()

df_deep_ssm = model_deep_ssm.transform(df_feat_raw[FEAT_FRACDIFF])
df_lg_ssm = model_lg_ssm.transform(df_feat_raw[FEAT_FRACDIFF])
df_feat_full = pd.concat([df_deep_ssm, df_lg_ssm, df_feat_raw], axis=1)[FEAT_L6]

test_features = df_feat_full[
    (df_feat_full.index.to_numpy() >= date_to_timestamp("2025-03-01")) &
    (df_feat_full.index.to_numpy() < date_to_timestamp("2025-08-31"))
    ]

candles = np.load("data/bar_deap_v1.npy")
timestamps = candles[:, 0].astype(np.int64)
mask = (timestamps >= test_features.index[0]) & (timestamps <= test_features.index[-1])

volume = candles[:, 5][mask]
datetimes = pd.to_datetime(test_features.index, unit="ms")

assert len(volume) == len(datetimes)

model = lgb.Booster(model_file="strategies/BinanceBtcDeapV1Voting/models/model_l6.txt")
res = model.predict(test_features)
assert len(res) == len(datetimes)

PyTorch configured: device=cpu, dtype=torch.float32


/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.12/site-packages/jesse/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.json
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.json


In [2]:
from research.plot import analyze_confidence_slices

analyze_confidence_slices(
    datetimes,
    res,
    volume,
    full_range=True,
)

已生成: conf_0.99_1.00_ratio_0.0077.jpg (置信度区间: [0.99, 1.00] (做多), 样本占比: 0.77%)
已生成: conf_0.98_0.99_ratio_0.0026.jpg (置信度区间: [0.98, 0.99] (做多), 样本占比: 0.26%)
已生成: conf_0.97_0.98_ratio_0.0000.jpg (置信度区间: [0.97, 0.98] (做多), 样本占比: 0.00%)
已生成: conf_0.96_0.97_ratio_0.0051.jpg (置信度区间: [0.96, 0.97] (做多), 样本占比: 0.51%)
已生成: conf_0.95_0.96_ratio_0.0051.jpg (置信度区间: [0.95, 0.96] (做多), 样本占比: 0.51%)
已生成: conf_0.94_0.95_ratio_0.0090.jpg (置信度区间: [0.94, 0.95] (做多), 样本占比: 0.90%)
已生成: conf_0.93_0.94_ratio_0.0103.jpg (置信度区间: [0.93, 0.94] (做多), 样本占比: 1.03%)
已生成: conf_0.92_0.93_ratio_0.0103.jpg (置信度区间: [0.92, 0.93] (做多), 样本占比: 1.03%)
已生成: conf_0.91_0.92_ratio_0.0051.jpg (置信度区间: [0.91, 0.92] (做多), 样本占比: 0.51%)
已生成: conf_0.90_0.91_ratio_0.0064.jpg (置信度区间: [0.90, 0.91] (做多), 样本占比: 0.64%)
已生成: conf_0.89_0.90_ratio_0.0039.jpg (置信度区间: [0.89, 0.90] (做多), 样本占比: 0.39%)
已生成: conf_0.88_0.89_ratio_0.0103.jpg (置信度区间: [0.88, 0.89] (做多), 样本占比: 1.03%)
已生成: conf_0.87_0.88_ratio_0.0064.jpg (置信度区间: [0.87, 0.88] (做多), 样本占比: 0.64%)

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data_1 = pd.read_csv('model_l4_scores.csv')          ############   测试文件

data_1_size = data_1.shape[0]     ###### 测试数据行数  ###############
m_size = 25     ####### 测试多少个月 #######
buy = 1     ##### 多 ###################
sell = 0     ##### 空 ####################
rrr = 0.25     ###### 系数 ###################
m = 1000     ###### 总资金 ###################

In [2]:
ft = 0.01   ### 0.1, 0.05, 0.01

if ft == 0.1:
    da = 1
    xiao = 0.9
    z = 5

if ft == 0.05:
    da = 1
    xiao = 0.95
    z = 10
    
if ft == 0.01:
    da = 1
    xiao = 0.99
    z = 50


for i in range(z):        

    data_1['final'] = 0

    mask = (data_1['score'] < da) & (data_1['score'] >= xiao)
    data_1['final'] = np.where(mask, data_1['volume'], 0)
    
    true_count = mask.sum() / data_1_size 
    
    ### 更新置信度
    da = round((da - ft), 2)
    xiao = round((xiao - ft), 2)

    data_1['high'] = data_1['final'].cumsum()
    data_1['open'] = rrr * data_1['high'] / m

    fig=plt.figure(figsize=(20,10))  
    plt.plot(data_1['eob'], data_1['open'])
    plt.xticks(range(0,data_1_size,int(data_1_size/m_size)))   
    fig.autofmt_xdate()
    plt.grid(1)
    plt.savefig("./temp/" + str(da) + "_" + str(true_count.round(2)) + ".jpg")   
    plt.close()